# Locality Sensitive Hashing (LSH)

| NOTATION      | DESCRIPTION |
| ----------- | ----------- |
| $d$      | dimension       |
| $\mathcal{D}$   | Distance function        |
| $\mathcal{S}$   | Similarity function       |
| $q$   | query object        |
| $q^*$   | object nearest to query        |
| $x$   | general object        |

In the previous lectures, we have seen how the problem of efficient exact-search (or related problems like set membership) etc can be solved using hash tables. Also, we saw how memory and computation efficient algorithms can be built if we are ready to sacrifice some accuracy like with bloom filters for set membership problem. However, a lot of practical applications demand for some tolerance in search. For example, while searching a data base of products for a specific product, say "nike shoes model no.831533-001", a user might trigger various queries with the same intention such as "nike shoes", "nike running shoes", "nike shoes latest", "nike shoes black" etc. The expectation is to recieve a list of products related to the query and not the exact query.

Abstractly,a relevant problem in information retrieval is that of "near neighbour search" or "finding elements from a set that are similar to a query element". Generally the notion of nearness (or corresponding similarity) can be specified using distance function, $\mathcal{D}$, ( or corresponding similarity function, $\mathcal{S}$). As a solution to this problem, we want to find the element that is closest (or most similar) to the query element according to the given function ($\mathcal{D}$ or $\mathcal{S}$. We formally define the problem of near neighbour search as follows.


## Near Neighbour Search (NN)

---
Given a collection of n objects, say $\mathcal{C}$ and a query object $q$ and a distance metric $\mathcal{D}$ ( or similarity metric $\mathcal{S}$), the near neighbour problem is to find the element in set $\mathcal{C}$ that is closest to $q$ according to $\mathcal{D}$ (or $\mathcal{S}$). Formally,

$$ q^* = \textrm{arg} \min_{x \in \mathcal{C}} \mathcal{D}(q, x) \quad \textrm{or} \quad q^* = \textrm{arg} \max_{x \in \mathcal{C}} \mathcal{S}(q, x) $$

---

<span style="color: red;">todo: does D have to be a metric? Does S have to be a positive semi-definite kernel? </span>
The similarity function, $\mathcal{S}(x,y)$ is also sometimes referred to as kernel function denoted by $k(x,y)$ or $\phi(x,y)$. A realted problem is that of finding the K near neighbours(KNN).


A naive algorithm to solve NN is to compute the distance $\mathcal{D}(x, q)$ for every x in $\mathcal{C}$ and find the element with minimum $\mathcal{D}(q,x)$. This is $\mathcal{O}(n)$ in computation of distance. One can also check that this is the lower bound on complexity as we have to compare the distances for all of the objects in $\mathcal{C}$. Another setup of the problem is that we have the set $\mathcal{C}$ fixed and we plan to make a lot of queries, $\{q_i\}_{i=1}^t$. In such a setup, it is wise to construct a datastructure which can help us answer these queries efficiently. When the objects are actually vectors in $R^d$ for some d, some space paritioning methods are used to create an efficient datastructure. <span style="color: red;">[cite algorithms] </span>. However, these datastructures suffer from the curse of dimensionality <span style="color: red;">[explain] </span>. and hence do not scale to higher dimensions. In order to find an efficient solution to the problem, we once again relax the problem in a way that it's practical utility is maintained. This problem is known as the Approximate Near Neighbour(ANN) search 

## Approximate Near Neighbour Search (ANN)

---
Given a collection of n objects, say $\mathcal{C}$ and a query object $q$, a distance metric $\mathcal{D}$ ( or similarity metric $\mathcal{S}$) and a error tolerance $\epsilon$, the approximate near neighbour problem is to find the element in set $\mathcal{C}$ that is within a distance of $(1+\epsilon)\mathcal{D}(q^*, q)$ where $q^*$ is the closest element

$$ q^+ \quad s.t. \quad \mathcal{D}(q^+,q) < (1 + \epsilon) \mathcal{D}(q^*,q) \quad \textrm{or} \quad \mathcal{S}(q^+, q) > (1-\epsilon) \mathcal{S}(q^*,q) $$

---

In an adaptation of the problem, we might be interested in all the points that lie within a distance from the query point q or the points which have a similarity greator than a particular threshold. 

> Note: In all the following material we will stick to the Similarity notion for near neighbour discussion. However, it should be noted that the two notions are inter-changeable.

## LSH - Introduction

Let us reflect on how we solved efficient search with hash-tables. Using a hash function, we can reduce the search space for a particular query by orders of magnitude. Specifically, we only have to compare the query against the elements that have same hash values and are stored together in memory. The hash function we used there was a random hash function as we had two conditions : (1) Each element x is consistently mapped to a location (which is a given since the mapping is a function) (2) Distinct elements collide as less as possible (as you cannot avoid collisions completely.) 

A key issue with using the same hash function as above for this problem is that the hash function will not distinguish between two elements y and z on the basis of how similar they are to x. All that matters for this hash function is whether two elements are exacty same or not. If we were to design hash function such that similar elements match together, then we can hope to efficiently solve the problem.